# Dynamisches Labyrinth

## TODOS

1.1 CSV einlesen

In [283]:
import csv

# Pfad zur CSV-Datei
csv_datei_pfad = './data/Puzzle_middle_3.csv'

# Liste zum Speichern der Daten
spielfeld = []

# CSV-Datei öffnen und Daten einlesen
with open(csv_datei_pfad, 'r') as csv_datei:
    csv_reader = csv.reader(csv_datei, delimiter=";")
    
    # Zeile für Zeile durch die CSV-Datei iterieren
    for zeile in csv_reader:
        spielfeld.append(zeile)

# Die 'daten'-Liste enthält nun die Daten aus der CSV-Datei
# daten[1] <- Spalte 1 mit Anführungszeichen
# daten[1][0] <- Spalte 1, zweites Array immer mit Index 0
# daten[1][0][2] <- Spalte 1, mit Index 2 gibt eine einzelene Zahl/Spielfeld
print(spielfeld)

[['2', '8', '0', '5'], ['1', '5', '8', '1'], ['0', '7', '7', '2'], ['6', '1', '9', '5'], ['4', '6', '1', '5'], ['2', '', '', '']]


## 2. Spielsteine definieren

Erklärung: Es gibt 10 verschiedene Spielsteine die angeben, in welche Richtung der Spieler sich bewegen und nicht bewegen kann. Dabei werden die verschiedenen Spielsteine in einer
- Matrix von 0 - 9 dargestellt und
- jeder Spielstein erhält zudem eine 2x2 Matrix der die möglichen Richtungen angibt. 

Dabei gilt folgendes: [ left, top, right, bottom] wobei eine 1 für eine Verbindung und eine 0 für keine Verbindung steht zum nächsten Feld steht. Als Beipsiel, [0, 0, 1, 1] steht für [-, -, right, bottom], also ist dieser Spielstein eine Ecke die eine Verbindung von unten nach rechts besitzt.

In [284]:
# [ left, top, right, bottom]
tile0 = [0, 0, 1, 1]
tile1 = [1, 0, 0, 1]
tile2 = [0, 1, 1, 0]
tile3 = [1, 1, 0, 0]
tile4 = [1, 0, 1, 1]
tile5 = [1, 1, 0, 1]
tile6 = [1, 1, 1, 0]
tile7 = [0, 1, 1, 1]
tile8 = [0, 1, 0, 1]
tile9 = [1, 0, 1, 0]

In [285]:
# Kodierung für das Spielfeld
tiles = [tile0, tile1, tile2, tile3, tile4, tile5, tile6, tile7, tile8, tile9]

Grid Klasse

In [286]:
# überarbeitet
class Grid:
    def __init__(self, src, dst, matrix):
        self.src = src
        self.dst = dst
        self.off_tile = int(matrix[-1][0])
        self.rows = len(matrix)-1
        self.cols = len(spielfeld[0]) if self.rows > 0 else 0 #self.cols = len(spielfeld[0][0].split(";")) if self.rows > 0 else 0
        self.nodes = [(i, j) for i in range(self.rows) for j in range(self.cols)]
        self.grid_values = [[0 for _ in range(self.cols)] for _ in range(self.rows)]

        for i in range(self.rows):
            for j in range(self.cols):
                self.grid_values[i][j] = int(matrix[i][j])

    def get_value(self, row, col):
        if 0 <= row < self.rows and 0 <= col < self.cols:
            return self.grid_values[row][col]
        return None  # Wert außerhalb der Grenzen des Grids
    
    def set_value(self, row, col, new_value):
        self.grid_values[row][col] = int(new_value)

    def move_player_left(self, player_src):
        row, col = player_src

        if col == 0:
            new_position = (row, self.cols - 1)  # Wrap around to the first column
        else:
            new_position = (row, (col - 1) % self.cols)  # Move to the next column within bounds

        return new_position
    
    def move_row_left(self, row_index, new_tile):
        row = self.grid_values[row_index]

        moved_out = row[0]  # Store the value that will be moved out of the row

        for i in range(1, self.cols):
            self.set_value(row_index, i - 1, row[i])

        self.set_value(row_index, self.cols - 1, new_tile)

        return moved_out
        
    def move_player_right(self, player_src):
        row, col = player_src

        if col == self.cols - 1:
            new_position = (row, 0)  # Wrap around to the first column
        else:
            new_position = (row, (col + 1) % self.cols)  # Move to the next column within bounds

        return new_position
    
    def move_row_right(self, row_index, new_tile):
        row = self.grid_values[row_index]

        moved_out = row[-1]  # Store the value that will be moved out of the row

        for i in range(self.cols - 1, 0, -1):
            self.set_value(row_index, i, row[i-1])

        self.set_value(row_index, 0, new_tile)

        return moved_out

    def get_nodes(self):
        return self.nodes
    
    def get_adjacent(self, node):       
        y, x = node
        adjacent_nodes = []

        # Horizontal Nachbarn
        for dx in [-1, 1]:
            new_x, new_y = x + dx, y
            # Wenn im Grid dann
            if 0 <= new_y < self.rows and 0 <= new_x < self.cols:
                if dx == -1:
                    possible_path_new_x = tiles[self.get_value(new_y, new_x)]
                    possible_path_x = tiles[self.get_value(y, x)]

                    # print(possible_path_new_x)
                    # print(possible_path_x)

                    if possible_path_new_x[2] == 1 and possible_path_x[0] == 1:                   
                        adjacent_nodes.append((new_y, new_x))

                if dx == 1:
                    possible_path_new_x = tiles[self.get_value(new_y, new_x)]
                    possible_path_x = tiles[self.get_value(y, x)]

                    # print(possible_path_new_x)
                    # print(possible_path_x)
                    if possible_path_new_x[0] == 1 and possible_path_x[2] == 1: 
                        adjacent_nodes.append((new_y, new_x))

        # Vertikal Nachbarn
        for dy in [-1, 1]:
            new_x, new_y = x, y + dy

            # print(dy)
            # print(new_y, new_x)
            if 0 <= new_y < self.rows and 0 <= new_x <= self.cols:
                if dy == -1:
                    possible_path_new_y = tiles[self.get_value(new_y, new_x)]
                    possible_path_y = tiles[self.get_value(y, x)]

                    # print("-1 new", possible_path_new_y)
                    # print("-1 ", possible_path_y)

                    if possible_path_new_y[3] == 1 and possible_path_y[1] == 1:
                        adjacent_nodes.append((new_y, new_x))


                if dy == 1:
                    possible_path_new_y = tiles[self.get_value(new_y, new_x)]
                    possible_path_y = tiles[self.get_value(y, x)]

                    # print("1 new", possible_path_new_y)
                    # print("1 ", possible_path_y)
                    # print("t:", self.get_value(y, x))

                    if possible_path_new_y[1] == 1 and possible_path_y[3] == 1:
                        adjacent_nodes.append((new_y, new_x))
                

        return adjacent_nodes

## Spielfeld Grid initialiseren aus spielfeld csv

In [287]:
spielfeld_grid = Grid((4,0), (0,3), spielfeld)
print(spielfeld_grid.get_nodes())
print("Nachbarn: ", spielfeld_grid.get_adjacent((1,2)))

[(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3), (3, 0), (3, 1), (3, 2), (3, 3), (4, 0), (4, 1), (4, 2), (4, 3)]
Nachbarn:  [(0, 2), (2, 2)]


## 3. A* anhand lösbarem Beispiel

Some text

In [288]:
# Replace this with your actual heuristic function
def heuristic_cost_estimate(current, goal, grid):
    # Example: Man
    x1, y1 = current
    x2, y2 = goal

    # Wert aus dem Grid mit x1, x2
    grid_tile_number = grid.get_value(x1,y1)
    
    # Tile bestimmen durch Wert aus Grid
    grid_tile_value = tiles[grid_tile_number]
    # Wenn Tile mit Weg nach oben +0 (Wird besser gewichtet)
    if grid_tile_value[1]:
        return abs(x2 - x1) + abs(y2 - y1)
        #return (((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5)
    
    # Wenn Tile nicht mit Weg nach oben +2
    return abs(x2 - x1) + abs(y2 - y1) + 2
    #return ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5 + 1

Zwei verschiedene Möglichkeiten das Grid zu verschieben (Also ein Spielstein von außen ins spielfeld schieben):
- Erstens durch Regression 
- Zweitens durch verschachtelte While Schleife

In [289]:
def a_star(grid):
    nodes = grid.get_nodes()
    closed_set = []
    open_set = [grid.src]
    prev = {node: None for node in nodes}
    g_score = {node: float("inf") for node in nodes}
    src = grid.src # das ist eigentlich unser Spieler
    dst = grid.dst
    g_score[src] = 0
    f_score = {node: float("inf") for node in nodes}
    f_score[src] = heuristic_cost_estimate(src, dst, grid)
    
    best_node_cost = f_score[src]
    best_node = grid.src

    # best_node ist auch grid.src
    while best_node != dst:
        # V1 um die while nochmal eine while sollange unser bestnode nicht u
        while open_set:
            u = min(open_set, key=f_score.get)
            #print("Node A*: ", u)
            open_set.remove(u)
            closed_set.append(u)
            
            # Ziel gefunden
            if u == dst:
                print('Found end')
                break

            # gibt es kein weg zum ziel dann:
                # Verschieben oder laufen?
                # - Position ermitteln laufen
                # - Position ermitteln nachdem schieben
                # Auswählen, welche Position besser ist und dahin platzieren

            # Nähstes Spielstein zum Ziel, verschieben
            else:
                print("Schritt")
                for v in grid.get_adjacent(u):
                    # print("closed: ", closed_set)
                    # print("Open: ", open_set)
                    new_g = g_score[u] + 1
                    
                    if v not in open_set and v not in closed_set:
                        open_set.append(v)
                        g_score[v] = new_g
                        f_score[v] = new_g + heuristic_cost_estimate(v, dst, grid)
                        # merke den knoten wo du stehtst (3, 0)
                        
                        # besten knoten im "subtree"
                        # wir wählen den erst Besten ("<") aufgrund der weiter benötigten Schritte (wenn kein Ende vorhanden)
                        if heuristic_cost_estimate(v, dst, grid) < best_node_cost:
                            best_node = v
                            best_node_cost = heuristic_cost_estimate(v, dst, grid)
                            # print("Best node: ", best_node)
                                       

                        prev[v] = u
                    elif ((v in open_set) or (v in closed_set)) and (new_g < g_score[v]):
                        g_score[v] = new_g
                        f_score[v] = new_g + heuristic_cost_estimate(v, dst, grid)
                        prev[v] = u
                        
                        if v in closed_set:
                            open_set.append(v)
                            closed_set.remove(v)

                        # besten knoten im "subtree"
                        # wir wählen den erst Besten ("<") aufgrund der weiter benötigten Schritte (wenn kein Ende vorhanden)
                        if heuristic_cost_estimate(v, dst, grid) < best_node_cost:
                            best_node = v
                            best_node_cost = heuristic_cost_estimate(v, dst, grid)
                            # print("Best node 2: ", best_node)
                        
            print("while openset")   
        

        # Ziel gefunden
        print("best_node", best_node)
        print("dst", dst)
        # ziel?
        if best_node == dst:      
            print(tiles[grid.get_value(dst[0],dst[1])][1])
            print('Found end')
            break
        
        # sonst verschieben


        # print("Off", grid.off_tile)
        # print("Best Node", best_node)

        # Alle geraden Reihen dann nach rechts
        if (best_node[0] % 2 == 0):
            grid.off_tile = grid.move_row_right(best_node[0], grid.off_tile)
            best_node = grid.move_player_right(best_node) 
        else: 
            grid.off_tile = grid.move_row_left(best_node[0], grid.off_tile)
            best_node = grid.move_player_left(best_node)

        # print("Off", grid.off_tile)
        # print("Best Node", best_node)

        # row holen
        # Karte holen
        # verschieben (gerade ungerade) => 
            # reihe verschieben mit neuer karte
            # spieler verschieben

        # am ende nachdem verschoebn wurde
        open_set = [best_node]
        
        print("while not dst")
    
    print("End")
    
    # Now reconstruct the path if it was found
    path = []
    if prev[dst] is not None:
        current = dst
        while current is not None:
            path.append(current)
            current = prev[current]
        path = path[::-1]  # Reverse to get the path from src to dst
    
    return path


In [290]:

optimal_path = a_star(spielfeld_grid)

Schritt
while openset
Schritt
while openset
Schritt
while openset
Schritt
while openset
Schritt
while openset
Schritt
while openset
best_node (4, 1)
dst (0, 3)
while not dst
Schritt
while openset
Schritt
while openset
best_node (4, 2)
dst (0, 3)
while not dst
Schritt
while openset
Schritt
while openset
Schritt
while openset
best_node (3, 3)
dst (0, 3)
while not dst
Schritt
while openset
Schritt
while openset
Schritt
while openset
Schritt
while openset
Found end
best_node (0, 3)
dst (0, 3)
1
Found end
End


## Grafische Ausgabe

In [291]:
string_map = {
    0: "┌",
    1: "┐",
    2: "└",
    3: "┘",
    4: "┬",
    5: "┤",
    6: "┴",
    7: "├",
    8: "|",
    9: "-",
}

In [292]:
def create_path_matrix(path, rows, cols, grid: Grid):
    optimal_path_matrix = [[' ' for _ in range(cols)] for _ in range(rows)]

    for node in path:
        y, x = node
        optimal_path_matrix[y][x] = string_map[grid.get_value(y,x)]  # Mark the cell in the path

    return optimal_path_matrix

path_matrix = create_path_matrix(optimal_path, spielfeld_grid.rows, spielfeld_grid.rows, spielfeld_grid)

print("Optimal Path")
# Gib die Matrix aus
for row in path_matrix:
    print(' '.join(row))


Optimal Path
    ┌ ┤  
    |    
    ├    
    ┤ ┐  
┤ └ ┬ ┴  


In [293]:
matrix = [
    [2, 8, 0, 5],
    [1, 5, 8, 1],
    [0, 7, 7, 2],
    [6, 1, 9, 5],
    [4, 6, 1, 5]
]

In [294]:
# # Beispiel
# # Spielfield hat bei 0;0 die Zahl 0, dadurch ergibt sich tiles[0] => [0, 0, 1, 1]
# #print(tiles[0])#
# print("Wert Spielstein", spielfeld_grid.get_value(0,1))
# print("Spielstein Kodierung: ", tiles[spielfeld_grid.get_value(0,1)])

# # Spielfeld Grid 
# print("Nodes: ", spielfeld_grid.get_nodes())
# test_node = (4,0)
# print("Nachbarn: ", spielfeld_grid.get_adjacent(test_node))
# print("Nachbarn: ", spielfeld_grid.get_adjacent((3,0)))
# print("Nachbarn: ", string_map[spielfeld_grid.get_value(4,0)])
# print("Anzahl Nachbarn", len(spielfeld_grid.get_adjacent(test_node)))
# print("Wert Node", spielfeld_grid.grid_values)
# print("Wert Node", tiles[spielfeld_grid.get_value(3,0)])
# print("Wert Node", tiles[spielfeld_grid.get_value(4,0)])

In [295]:
# Create a matrix of strings
string_matrix = [[string_map[num] for num in row] for row in matrix]

# Display the string matrix
for row in string_matrix:
    print(" ".join(row))

└ | ┌ ┤
┐ ┤ | ┐
┌ ├ ├ └
┴ ┐ - ┤
┬ ┴ ┐ ┤


3. A* anhand lösbarem Beispiel

- checken grenzen
    - wenn laufen nicht möglich, dann schieben
        
- verschieben  
    - Zeile über Spielfigur in möglich Richtung
    
- checken funktion/algo
- Listen führen: Open/Close

- Heuristik:
    - 1 Schritt = +1
    - 1 Verschieben = +1
    - Heuristischer wert = Schritte bis zum Ziel
        -   ABER wenn nicht Öffnung nach oben +2

WAS TUN BEI GLEICHSTAND?
AB WANN VERSCHIEBEN (auch wenn laufen möglich)

- jeden möglichen weg berechnen
- jeden möglichen weg nach verschiebungen